# Quant AALTD Benchmark

Simple benchmark notebook to test the quant AALTD algorithm.
- Algorithm: QuantAALTD2024
- Metrics: Runtime (train/predict), accuracy, memory usage
- Dataset: Configurable via DATASET_NAME variable

In [ ]:
# CONFIGURATION - Change this to test different datasets
DATASET_NAME = "Pedestrian"  # Change this to any UCR dataset name
TRAIN_PCT = 1.0  # Percentage of training data to use
TEST_PCT = 1.0   # Percentage of test data to use
NUM_ESTIMATORS = 200  # Number of estimators for QuantAALTD2024

In [ ]:
# Setup paths BEFORE importing tsckit
import sys

ROOT_DIR = '/Users/urav/code/research'

# Add algorithm code paths
sys.path.extend([
    f'{ROOT_DIR}',                    # For tsckit package
    f'{ROOT_DIR}/quant/code',         # For original quant.py
    f'{ROOT_DIR}/hydra/code',         # For original hydra.py
    f'{ROOT_DIR}/aaltd2024/code',     # For quant_aaltd.py, hydra_gpu.py, utils.py, ridge.py
])


print("✓ Paths configured")

In [ ]:
# Import libraries
import time
import psutil
import numpy as np
from tsckit import MonsterDataset, QuantAALTD2024

print("✓ Libraries imported")

In [ ]:
# Memory usage helper
def get_memory_usage():
    return psutil.Process().memory_info().rss / 1024 / 1024  # MB

print(f"Initial memory usage: {get_memory_usage():.1f} MB")

In [ ]:
# Load dataset
print(f"Loading {DATASET_NAME} dataset...")
dataset = MonsterDataset(DATASET_NAME, fold=0, train_pct=TRAIN_PCT, test_pct=TEST_PCT)
print(f"Dataset: {dataset.info()}")

# Get test labels for accuracy calculation
_, y_test = dataset.get_arrays("test")
print(f"Test samples: {len(y_test)}, Classes: {len(np.unique(y_test))}")

In [ ]:
# Initialize QuantAALTD2024 model
print("Initializing QuantAALTD2024...")
model = QuantAALTD2024(num_estimators=NUM_ESTIMATORS)
print(f"Model: {model.name}")

memory_after_init = get_memory_usage()
print(f"Memory after initialization: {memory_after_init:.1f} MB")

In [ ]:
# Training phase
print("\n=== TRAINING PHASE ===")
memory_before_train = get_memory_usage()
train_start_time = time.time()

model.fit(dataset)

train_end_time = time.time()
memory_after_train = get_memory_usage()

train_time = train_end_time - train_start_time
train_memory_delta = memory_after_train - memory_before_train

print(f"Training time: {train_time:.3f} seconds")
print(f"Memory usage during training: +{train_memory_delta:.1f} MB")
print(f"Total memory after training: {memory_after_train:.1f} MB")

In [ ]:
# Prediction phase
print("\n=== PREDICTION PHASE ===")
memory_before_pred = get_memory_usage()
pred_start_time = time.time()

predictions = model.predict(dataset)

pred_end_time = time.time()
memory_after_pred = get_memory_usage()

pred_time = pred_end_time - pred_start_time
pred_memory_delta = memory_after_pred - memory_before_pred

print(f"Prediction time: {pred_time:.3f} seconds")
print(f"Memory usage during prediction: +{pred_memory_delta:.1f} MB")
print(f"Total memory after prediction: {memory_after_pred:.1f} MB")

In [ ]:
# Results evaluation
print("\n=== RESULTS ===")
accuracy = np.mean(predictions == y_test)

print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Predictions shape: {predictions.shape}")
print(f"Correct predictions: {np.sum(predictions == y_test)} / {len(y_test)}")

In [ ]:
# Summary report
print("\n=== BENCHMARK SUMMARY ===")
print(f"Algorithm: {model.name}")
print(f"Dataset: {DATASET_NAME} (Train: {len(dataset.train_indices)}, Test: {len(y_test)})")
print(f"")
print(f"Performance:")
print(f"  - Training time: {train_time:.3f}s")
print(f"  - Prediction time: {pred_time:.3f}s")
print(f"  - Total time: {train_time + pred_time:.3f}s")
print(f"  - Accuracy: {accuracy:.4f}")
print(f"")
print(f"Memory usage:")
print(f"  - Training: +{train_memory_delta:.1f} MB")
print(f"  - Prediction: +{pred_memory_delta:.1f} MB")
print(f"  - Peak: {memory_after_pred:.1f} MB")